####  <span style="color:#5D8BF4"> Irena Vent </span>

# <span style="color:#051367"> Machine Learning</span>

<span style="color:#051367"> **Dataset**: Airbnb </span>

<span style="color:#051367"> **Objetivo**: Predecir el precio de un airbnb situado Madrid</span>


In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

import warnings
warnings.filterwarnings('ignore')

In [3]:
def plot_confusion_matrix(confmat):
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.5)
    for i in range(confmat.shape[0]):
        for j in range(confmat.shape[1]):
            ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')

    plt.xlabel('predicted label')
    plt.ylabel('true label')

    plt.tight_layout()
    plt.show()

 <span style="color:#5D8BF4"> Cargamos nuestro dataset con el objetivo de: </span>
    
1. Filtrar el dataset por Madird (Spain); 
2. Comprobar si existen variables que puedan ser descartadas previo al análisis exploratior de datos, por ejemplo: ids, urls, etc.

In [4]:
prueba = pd.read_csv('./data/airbnb-listings.csv', sep=';', decimal='.')

#pd.set_option('display.max_columns', None)
#prueba.head()

In [5]:
prueba.shape

(14780, 89)

In [6]:
# eliminamos las muestras que contienen NaN en las columnas indicadas
prueba = prueba.dropna(subset=['Country', 'City'])

# filtramos por Spian y Madrid
prueba = prueba.loc[prueba['Country'] == 'Spain']
prueba = prueba[prueba["City"].str.contains("Madrid")]

#print(f'Dimensión dataframe --> {prueba.shape}')
#print(f'Valores únicos en Country --> {prueba["Country"].unique()}')
#print(f'Valores únicos en City --> {prueba["City"].unique()}')

# eliminamos columnas
prueba = prueba.drop(['ID', 'Listing Url', 'Scrape ID', 'Last Scraped', 'Name', 'Summary', 'Space', 'Description',
                      'Experiences Offered', 'Neighborhood Overview','Zipcode' , 'Notes', 'Transit', 'Access', 
                      'Interaction', 'House Rules', 'Thumbnail Url', 'Medium Url', 'Picture Url', 'XL Picture Url',
                      'Host ID', 'Host URL', 'Host Name', 'Host Since', 'Host Location', 'Host About','Amenities', 
                      'Host Thumbnail Url', 'Host Picture Url', 'Host Neighbourhood','Host Listings Count', 
                      'Host Total Listings Count', 'Host Verifications', 'Street', 'City', 'State', 'Market',
                      'Smart Location', 'Country Code', 'Country', 'First Review', 'Last Review', 'Calendar Updated', 
                      'Calendar last Scraped', 'License', 'Jurisdiction Names', 'Calculated host listings count',
                      'Geolocation', 'Features'], axis = 1)

prueba.shape

(13234, 40)

<span style="color:#5D8BF4"> **Divsión del dataset en train y test** para el desarrollo de análisis exploratorio de datos. Que se realizará únicamente sobre el subconjunto de train. Posteriormente, en la validación del modelo elegido, las decisiones tomadas sobre train se aplicaran al subconjunto de test. </span> 

In [7]:
from sklearn.model_selection import train_test_split

#full_df = pd.read_csv('./airbnb-listings-extract.csv', sep=';', decimal='.')
train, test = train_test_split(prueba, test_size=0.2, shuffle=True, random_state=0)

print(f'Dimensiones del dataset de training: {train.shape}')
print(f'Dimensiones del dataset de test: {test.shape}')

# Guardamos
train.to_csv('./train.csv', sep=';', decimal='.', index=False)
test.to_csv('./test.csv', sep=';', decimal='.', index=False)

# A partir de este momento cargamos el dataset de train y trabajamos ÚNICAMENTE con él. 
df_train = pd.read_csv('./train.csv', sep=';', decimal='.')

Dimensiones del dataset de training: (10587, 40)
Dimensiones del dataset de test: (2647, 40)


<span style="color:#5D8BF4"> **Análisis exploratior de los datos** </span> 

In [ ]:
pd.set_option('display.max_columns', None)
prueba.head()

In [ ]:
df_train.describe().T

In [ ]:
#df_train.isnull().any()
df_train.isnull().sum()

In [ ]:
df_train.dtypes

<span style="color:#5D8BF4"> **Primeras observaciones**</span>

- Existen varibles, como son: Host Response Time, Neighbourhood, Neighbourhood Cleansed, Neighbourhood Group Cleansed, Property Type, Bed Type, Amenities, y Cancellation Policy, todas ellas de tipo **object** que deben ser transformadas y/o categorizadas;

- Las variables Host Acceptance Rate y Has Availability pueden ser eliminadas al tener todas sus muestras ausentes; 

- La variable objetivo Price tiene 9 muestras con valores ausentes, que se eliminarán;
       
- La varieble Square Feet debe ser transformada a Square Meter observamos que más de 96% de sus muestras están ausentes, por lo que decidimos eliminar dicha columna;
    
- La varible Zipcode parece tener datos erróneos, reisar y eliminar valores ausente;
    
- Imputar los valores ausentes con la moda en Bed, Bedroom y Bathroom, el 75% de la muestra está dentro del rango, pues parece haber algún autlier;
    
- Sobre las variables Neighbourhood, Neighbourhood Cleansed y Neighbourhood Group Cleansed ver contenido de cada varaibles para decidir si es necesario eliminar alguna de ellas;
    
- Para las variables Weekly Price y Monthly Price analizar con un estudio de correlación y decidir si se pueden eliminar;
    
- Muchas de las variabbles (Accommodates, Square Feet, Weekly Price, Monthly Price, Security Deposit, Number of Reviews, etc) presentan valores dispares, por lo que sería necesario explorar los datos para ecnontrar posibles outliers;
    
- Todas las variables Review excepto Number of Reviews, presentan valores ausentes, la imputación según los datos, puede ser a través de las media o valor meas frecuente. Pero sería necesario valorar la opción de eliminar alguna de dichas variables con un estudio de correlaciones; </span>

<span style="color:#5D8BF4"> **Gráfico de correlaciones** valorar si podemos eliminar alguna variables más en este punto.</span>

In [ ]:
import seaborn as sns

# Compute the correlation matrix
corr = np.abs(df_train.corr())

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(12, 10))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask,vmin = 0.0, vmax=1.0, center=0.5,
            linewidths=.1, cmap="YlGnBu", cbar_kws={"shrink": .8})

plt.show()

In [ ]:
# eliminamos las columnas con valores ausentes y alta correlación entre variables predictoras
# evitar la colinealidad
df_train = df_train.drop(['Host Acceptance Rate', 'Host Response Time', 'Has Availability', 'Square Feet', 
                          'Neighbourhood', 'Neighbourhood Cleansed', 'Weekly Price', 'Monthly Price', 
                          'Has Availability', 'Availability 30', 'Availability 60', 'Availability 365',
                          'Review Scores Accuracy', 'Review Scores Checkin','Review Scores Communication', 
                          'Reviews per Month', 'Review Scores Value'], axis = 1)

# eliminamos 9 muestras con Price NaN
df_train = df_train.dropna(subset=['Price'])

print(f'Porcentaje de muestras eliminadas: {round((10587-df_train.shape[0])/10587*100,4)} %')
print(f'Tamaño df_train: {df_train.shape}')